In [1]:
pip install transformers datasets evaluate jiwer

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset, Audio

minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
minds = minds.train_test_split(test_size=0.2)

In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 80
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 20
    })
})

In [5]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [6]:
minds["train"][0]

{'path': '/home/cgb3/.cache/huggingface/datasets/downloads/extracted/75236ad3a9fd9bbe635f6aa63106ed2d33d9507d30786db63f62cb59a5716646/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
 'audio': {'path': '/home/cgb3/.cache/huggingface/datasets/downloads/extracted/75236ad3a9fd9bbe635f6aa63106ed2d33d9507d30786db63f62cb59a5716646/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
  'array': array([ 0.00024414, -0.00024414,  0.        , ...,  0.01037598,
          0.0098877 ,  0.0098877 ]),
  'sampling_rate': 8000},
 'transcription': 'what is my account balance'}

In [7]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/configuration_utils.py:302: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

{'path': '/home/cgb3/.cache/huggingface/datasets/downloads/extracted/75236ad3a9fd9bbe635f6aa63106ed2d33d9507d30786db63f62cb59a5716646/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
 'audio': {'path': '/home/cgb3/.cache/huggingface/datasets/downloads/extracted/75236ad3a9fd9bbe635f6aa63106ed2d33d9507d30786db63f62cb59a5716646/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
  'array': array([ 2.35084444e-04,  3.73781659e-05, -2.34791543e-04, ...,
          1.11566903e-02,  9.70520079e-03,  4.98228893e-03]),
  'sampling_rate': 16000},
 'transcription': 'what is my account balance'}

In [9]:
def uppercase(example):
    return {"transcription": example["transcription"].upper()}


minds = minds.map(uppercase)

Map: 100%|██████████| 20/20 [00:00<00:00, 5103.49 examples/s]


In [10]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    batch["input_length"] = len(batch["input_values"][0])
    return batch

In [11]:
encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

Map (num_proc=4): 100%|██████████| 20/20 [00:00<00:00, 177.41 examples/s]


In [12]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")

In [25]:
import evaluate

#wer = evaluate.load("wer")

In [42]:
import numpy as np


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = evaluate.load("wer")
    #wer = 

    return {"wer": wer.compute(predictions=pred_str, references=label_str)}

In [43]:
from transformers import AutoModelForCTC, TrainingArguments, Trainer

model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/transformers/configuration_utils.py:302: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
training_args = TrainingArguments(
    output_dir="my_awesome_asr_mind_model",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=200,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Wer
100,33.995700,29.663630,1.000000
200,6.715300,5.154046,1.000000


/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=200, training_loss=27.734552917480467, metrics={'train_runtime': 160.7855, 'train_samples_per_second': 19.902, 'train_steps_per_second': 1.244, 'total_flos': 3.839687518470912e+17, 'train_loss': 27.734552917480467, 'epoch': 40.0})

In [45]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", "en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [50]:
from transformers import pipeline

transcriber = pipeline("automatic-speech-recognition", model="my_awesome_asr_mind_model/checkpoint-200")
transcriber(audio_file)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'text': ''}